# WCA - Resumen del 2024


In [ ]:
# Imports

from pathlib import Path

import pandas as pd
from sqlalchemy import create_engine

In [ ]:
## Database

user = "user"
password = "pass"
host = "127.0.0.1"
port = 3306
database = "wca_db"

conn_str = f"mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(conn_str)

## Reportes


In [ ]:
# Global Variables

COUNTRY = "Colombia"
YEAR = 2025

In [ ]:
def process_report(name: str, query: str) -> pd.DataFrame:
    df = pd.read_sql_query(query, engine)

    dst_file = Path(f"output/{COUNTRY}/{name}.csv")
    dst_file.parent.mkdir(parents=True, exist_ok=True)

    df.to_csv(dst_file, sep=";", index=False)
    print("Results saved in", dst_file)

    return df

### 1. Top de competidores con más podios


In [ ]:
df = process_report(
    "1 - Top de competidores con más podios",
    f"""
        select r.personName nombre, count(*) conteo
        from Results r
        join Competitions c on r.competitionId = c.id
        where 1=1
        and c.year = {YEAR}
        -- and c.countryId = '{COUNTRY}'
        and r.personCountryId = '{COUNTRY}'
        and r.roundTypeId in ('f', 'b', 'c')
        and r.pos in ('1', '2', '3')
        group by 1
        order by 2 desc
        limit 50
    """,
)
df.head(10)

### 2. Top de competidores con más podios en una competencia


In [ ]:
df = process_report(
    "2 - Top de competidores con más podios en una competencia",
    f"""
        with p1 as (
            select
                r.personName nombre,
                c.name competencia,
                c.month mes,
                c.day dia,
                count(*) conteo
            from Results r
            join Competitions c on r.competitionId = c.id
            where 1=1
            and c.year = {YEAR}
            -- and c.countryId = '{COUNTRY}'
            and r.personCountryId = '{COUNTRY}'
            and r.roundTypeId in ('f', 'b', 'c')
            and r.pos in ('1', '2', '3')
            group by 1, 2
        ), p2 as (
            select *, row_number() over (partition by nombre order by conteo desc, mes, dia) rn
            from p1
        )
        select p2.nombre, p2.competencia, p2.conteo
        from p2 where rn = 1
        order by 3 desc, p2.mes, p2.dia
        limit 50
    """,
)
df.head(10)

### 3. Top de competidores con más oros


In [ ]:
df = process_report(
    "3 - Top de competidores con más oros",
    f"""
        select r.personName nombre, count(*) conteo
        from Results r
        join Competitions c on r.competitionId = c.id
        where 1=1
        and c.year = {YEAR}
        -- and c.countryId = '{COUNTRY}'
        and r.personCountryId = '{COUNTRY}'
        and r.roundTypeId in ('f', 'b', 'c')
        and r.pos = '1'
        group by 1
        order by 2 desc
        limit 50
    """,
)
df.head(10)

### 4. Top de competidores con más oros en una competencia


In [ ]:
df = process_report(
    "4 - Top de competidores con más oros en una competencia",
    f"""
        with o1 as (
            select
                r.personName nombre,
                c.name competencia,
                c.month mes,
                c.day dia,
                count(*) conteo
            from Results r
            join Competitions c on r.competitionId = c.id
            where 1=1
            and c.year = {YEAR}
            -- and c.countryId = '{COUNTRY}'
            and r.personCountryId = '{COUNTRY}'
            and r.roundTypeId in ('f', 'b', 'c')
            and r.pos = '1'
            group by 1, 2
        ), o2 as (
            select *, row_number() over (partition by nombre order by conteo desc, mes, dia) rn
            from o1
        )
        select o2.nombre, o2.competencia, o2.conteo
        from o2 where rn = 1
        order by 3 desc, o2.mes, o2.dia
        limit 50
    """,
)
df.head(10)

### 5. Top PPT ponderado


In [ ]:
df = process_report(
    "5 - Top PPT ponderado",
    f"""
        with pods as (
            select
                r.personName nombre,
                count(case when r.pos = '1' then 1 else null end) as oros,
                count(case when r.pos = '2' then 1 else null end) as platas,
                count(case when r.pos = '3' then 1 else null end) as bronces,
                sum(4-r.pos) puntos
            from Results r
            join Competitions c on r.competitionId = c.id
            where 1=1
            and c.year = {YEAR}
            -- and c.countryId = '{COUNTRY}'
            and r.personCountryId = '{COUNTRY}'
            and r.roundTypeId in ('f', 'b', 'c')
            and r.pos in ('1', '2', '3')
            and r.best != '-1'
            group by 1
        ), comps as (
            select r.personName, count(distinct r.competitionId) cnt
            from Results r
            join Competitions c on r.competitionId = c.id
            where 1=1
            and c.year = {YEAR}
            -- and c.countryId = '{COUNTRY}'
            and r.personCountryId = '{COUNTRY}'
            group by 1
        )
        select p.*, c.cnt comps, p.puntos/c.cnt ppt
        from pods p
        join comps c on p.nombre = c.personName
        where c.cnt > 3
        order by 7 desc
        limit 50
    """,
)
df.head(10)

### 6. Top de competidores con más competencias


In [ ]:
df = process_report(
    "6 - Top de competidores con más competencias",
    f"""
        select r.personName nombre, count(distinct r.competitionId) conteo
        from Results r
        join Competitions c on r.competitionId = c.id
        where 1=1
        and c.year = {YEAR}
        -- and c.countryId = '{COUNTRY}'
        and r.personCountryId = '{COUNTRY}'
        group by 1
        order by 2 desc
        limit 50
    """,
)
df.head(10)

### 7. Top de competidores con más ciudades visitadas


In [ ]:
df = process_report(
    "7 - Top de competidores con más ciudades visitadas",
    f"""
        with cities as (
            select distinct
                r.personName,
                c.cityName,
                case
                    when c.cityName in ('Armenia') then 'Armenia, Quindío'
                    when c.cityName in ('Barrancabermeja') then 'Barrancabermeja, Santander'
                    when c.cityName in ('Bogotá', 'Bogotá D.C.', 'Bogotá, Cundinamarca') then 'Bogotá, Cundinamarca'
                    when c.cityName in ('Bucaramanga') then 'Bucaramanga, Santander'
                    when c.cityName in ('Cartagena') then 'Cartagena, Bolivar'
                    when c.cityName in ('Cartago') then 'Cartago, Valle del Cauca'
                    when c.cityName in ('Chocontá') then 'Chocontá, Cundinamarca'
                    when c.cityName in ('El Carmen De Viboral') then 'El Carmen de Viboral, Antioquia'
                    when c.cityName in ('Facatativá') then 'Facatativá, Cundinamarca'
                    when c.cityName in ('Florencia') then 'Florencia, Caquetá'
                    when c.cityName in ('Floridablanca') then 'Floridablanca, Santander'
                    when c.cityName in ('Funza') then 'Funza, Cundinamarca'
                    when c.cityName in ('Fusagasugá') then 'Fusagasugá, Cundinamarca'
                    when c.cityName in ('La Estrella') then 'La Estrella, Antioquia'
                    when c.cityName in ('La Sierra, Quipile, Cundinamarca') then 'La Sierra, Cundinamarca'
                    when c.cityName in ('Medellín - Antioquia', 'Medellin') then 'Medellín, Antioquia'
                    when c.cityName in ('Melgar') then 'Melgar, Tolima'
                    when c.cityName in ('Mosquera') then 'Mosquera, Cundinamarca'
                    when c.cityName in ('Neiva') then 'Neiva, Huila'
                    when c.cityName in ('Pereira') then 'Pereira, Risaralda'
                    when c.cityName in ('Quimbaya') then 'Quimbaya, Quindío'
                    when c.cityName in ('Ricaurte') then 'Ricaurte, Cundinamarca'
                    when c.cityName in ('San Gil') then 'San Gil, Santander'
                    when c.cityName in ('Tocancipá, Cundinamarca, Colombia') then 'Tocancipá, Cundinamarca'
                    when c.cityName in ('Tunja') then 'Tunja, Boyacá'
                    when c.cityName in ('Villavicencio') then 'Villavicencio, Meta'
                    when c.cityName in ('Yopal') then 'Yopal, Casanare'
                    when c.cityName in ('Zipaquirá') then 'Zipaquirá, Cundinamarca'
                    else replace(c.cityName, '.', '')
                end as normalized_city
            from Results r
            join Competitions c on r.competitionId = c.id
            where 1=1
            and c.cityName not in ('Multiple cities', 'Multiple locations')
            and c.year = {YEAR}
            -- and c.countryId = '{COUNTRY}'
            and r.personCountryId = '{COUNTRY}'
            order by normalized_city
        )
        select
            personName nombre,
            count(distinct normalized_city) conteo
        from cities
        group by 1
        order by 2 desc
        limit 50
    """,
)
df.head(10)

### 8. Número de novatos en el año


In [ ]:
df = process_report(
    "8 - Numero de novatos",
    f"""
        select
            substring(p.id, 1, 4) año,
            count(*) conteo
        from Persons p
        where 1=1
        and p.countryId = '{COUNTRY}'
        and substring(p.id, 1, 4) > '{YEAR - 5}'
        group by 1
    """,
)
df.head(10)


### 9. Top de novatos más rápidos en 333


In [ ]:
df = process_report(
    "9 - Top de novatos más rápidos en 333",
    f"""
        with n as (
            select
                r.personId,
                r.personName,
                c.name,
                c.cityName,
                c.countryId,
                r.average,
                rank() over (partition by r.personId order by c.year, c.month, c.day, r.average) rnk
            from Results r
            join Competitions c on c.id = r.competitionId
            where 1=1
            and r.personId like '{YEAR}%'
            and r.personCountryId = '{COUNTRY}'
            and r.eventId = '333'
            and r.average != -1
        )
        select
            n.personId wca_id,
            n.personName nombre,
            n.name torneo,
            n.cityName ciudad,
            n.countryId pais,
            n.average avg
        from n
        where n.rnk = 1
        order by n.average
        limit 50
    """,
)
df.head(10)

### 10. Top torneos con más novatos


In [ ]:
df = process_report(
    "10 - Top torneos con más novatos",
    f"""
        with n as (
            select
                c.name,
                c.cityName,
                r.personId,
                rank() over (partition by r.personId order by c.year, c.month, c.day) rnk
            from Results r
            join Competitions c on c.id = r.competitionId
            where 1=1
            and r.personId like '{YEAR}%'
            and c.countryId = '{COUNTRY}'
        ), nc as (
            select distinct *
            from n
            where rnk = 1
        )
        select
            name competencia,
            cityName ciudad,
            count(*) conteo
        from nc
        group by 1
        order by 3 desc
        limit 50
    """,
)
df.head(10)

### 11. Records del año


In [ ]:
df = process_report(
    "11 - Records del año",
    f"""
        with r as (
            select
                r.*,
                case
                    when regionalSingleRecord is not null and regionalAverageRecord is not null then 2
                    when regionalSingleRecord is not null then 1
                    when regionalAverageRecord is not null then 1
                    else 0
                end records
            from Results r
            join Competitions c on c.id = r.competitionId
            where 1=1
            and (r.regionalSingleRecord is not null or r.regionalAverageRecord is not null)
            and r.personCountryId = '{COUNTRY}'
            and c.year = {YEAR}
        )
        select personName nombre, sum(records) conteo
        from r
        group by 1
        order by 2 desc
    """,
)
df.head(10)

### 12. Top mujeres 333


In [ ]:
df = process_report(
    "12 - Top mujeres 333",
    f"""
        with f as (
            select *
            from Persons p
            where 1=1
            and p.gender = 'f'
            and p.countryId = '{COUNTRY}'
        ), r as (
            select r.personId, min(r.average) avg
            from Competitions c
            join Results r on r.competitionId = c.id
            where 1=1
            and r.average != -1
            and c.year = {YEAR}
            and r.eventId = '333'
            and r.personCountryId = '{COUNTRY}'
            group by 1
        )
        select f.id wca_id, f.name nombre, r.avg
        from r
        join f on f.id = r.personId
        order by 3
        limit 50
    """,
)
df.head(10)

In [ ]:
!jupyter nbconvert --to html wca_eoy.ipynb;